# Fleet-Level Analysis with NeuralMOVES

NeuralMOVES provides per-second, per-vehicle CO₂ emission estimates. But for policy analysis, you often need answers to bigger questions:

- *What is the fleet-average emission rate for the current US vehicle mix?*
- *How much CO₂ does the national passenger fleet emit annually?*
- *What would happen if EV adoption went from 5% to 30% over the next decade?*
- *How do emissions differ between Michigan (cold winters) and Florida (warm year-round)?*

NeuralMOVES v0.4.0 adds six **aggregation layers** that answer these questions by scaling microscopic results up to fleet averages, annual totals, and multi-year scenarios — using the same default data as EPA MOVES.

This notebook walks through each layer with working examples.

> **Prerequisite**: For per-second usage and basic API examples, see the [comprehensive usage guide](comprehensive_usage_guide.ipynb). This notebook focuses on the fleet and policy analysis capabilities.

In [ ]:
import neuralmoves
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(f"NeuralMOVES version: {neuralmoves.__version__}")

---
## 1. Drive Cycles (Layer 1)

A **drive cycle** is a speed trace over time — it represents how a vehicle is driven. Instead of calling `estimate_running_co2()` for each second and summing manually, Layer 1 wraps this into a single call that returns total CO₂, g/mile, and the full per-second timeseries.

NeuralMOVES provides `DriveCycle`, a lightweight container for speed profiles, with several ways to create one.

In [ ]:
from neuralmoves import DriveCycle, evaluate_cycle

# Create a constant-speed cycle at 30 mph for 5 minutes
# (includes a short ramp-up and ramp-down)
cycle_30 = DriveCycle.constant_speed(speed_mph=30, duration_s=300)
print(cycle_30)

# Create a highway cycle at 60 mph
cycle_60 = DriveCycle.constant_speed(speed_mph=60, duration_s=300)
print(cycle_60)

You can also create cycles from speed arrays (e.g., output from a traffic simulator):

In [ ]:
# Simulate a stop-and-go urban profile
speeds_mph = np.concatenate([
    np.linspace(0, 35, 20),   # accelerate
    np.full(30, 35),           # cruise
    np.linspace(35, 0, 15),    # brake
    np.zeros(10),              # stop
] * 4)  # repeat 4 times

urban_cycle = DriveCycle.from_speed_array(speeds_mph, name="Urban stop-and-go")
print(urban_cycle)

### Evaluating a cycle

`evaluate_cycle()` runs the neural network for every second of the cycle and returns summary metrics:

In [ ]:
# Evaluate the 30 mph cycle for a 2019 Gasoline Passenger Car
result = evaluate_cycle(
    cycle_30,
    temp=25, humid_pct=50,
    model_year=2019, source_type='Passenger Car', fuel_type='Gasoline',
)

print(f"Total CO\u2082: {result.total_co2_g:.1f} g")
print(f"Rate: {result.rate_g_per_mile:.1f} g/mile")
print(f"Distance: {result.distance_miles:.2f} miles")
print(f"Duration: {result.duration_s:.0f} seconds")

### Comparing across vehicle types

`evaluate_cycle_multi()` evaluates the same cycle for multiple vehicle configurations at once:

In [ ]:
from neuralmoves.cycles import evaluate_cycle_multi

configs = [
    {'model_year': 2019, 'source_type': 'Passenger Car', 'fuel_type': 'Gasoline'},
    {'model_year': 2019, 'source_type': 'Passenger Car', 'fuel_type': 'Diesel'},
    {'model_year': 2019, 'source_type': 'Passenger Truck', 'fuel_type': 'Gasoline'},
    {'model_year': 2019, 'source_type': 'Transit Bus', 'fuel_type': 'Diesel'},
    {'model_year': 2010, 'source_type': 'Passenger Car', 'fuel_type': 'Gasoline'},
]

comparison = evaluate_cycle_multi(cycle_30, temp=25, humid_pct=50, configs=configs)
print(comparison[['source_type', 'fuel_type', 'model_year', 'rate_g_per_mile']].to_string(index=False))

In [ ]:
# Visualize the comparison
fig, ax = plt.subplots(figsize=(10, 4))
labels = [f"{r['source_type']}\n{r['fuel_type']} {r['model_year']}" for _, r in comparison.iterrows()]
bars = ax.barh(labels, comparison['rate_g_per_mile'], color='steelblue')
ax.set_xlabel('CO\u2082 Emission Rate (g/mile)', fontsize=11)
ax.set_title('Per-Vehicle Emission Rates at 30 mph', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')
for bar, val in zip(bars, comparison['rate_g_per_mile']):
    ax.text(val + 5, bar.get_y() + bar.get_height()/2, f'{val:.0f}', va='center', fontsize=10)
plt.tight_layout()
plt.show()

---
## 2. Fleet Composition (Layer 2)

A single vehicle's emission rate is interesting, but policy questions require a **fleet average**: the weighted mean across all vehicle types, model years, and fuel types currently on the road.

NeuralMOVES builds the fleet mix from the same three MOVES default tables that EPA uses:
- **Age distribution**: What fraction of passenger cars are 0 years old, 1 year old, 2 years old, ...?
- **Fuel type mix**: For each (source type, model year), what share is gasoline vs. diesel vs. electric?
- **Population weights**: How many passenger cars vs. trucks vs. buses are on the road?

These are cross-multiplied to produce a fleet fraction for each vehicle configuration.

In [ ]:
from neuralmoves import FleetComposition

# Build the default US fleet for calendar year 2019
fleet_2019 = FleetComposition.from_moves_defaults(calendar_year=2019)
print(fleet_2019)
print(f"\nNumber of unique vehicle configs: {fleet_2019.n_vehicle_configs}")
print(f"Default EV penetration: {fleet_2019.ev_penetration:.2%}")

# Show fleet summary by source type
print("\nFleet composition by source type:")
print(fleet_2019.summary().to_string(index=False))

### Fleet-average emission rate

`fleet_average_rate()` evaluates the drive cycle for every vehicle config in the fleet and computes the weighted average. This typically involves ~99 NN evaluations and takes about 0.2 seconds.

In [ ]:
from neuralmoves import fleet_average_rate

cycle = DriveCycle.constant_speed(speed_mph=30, duration_s=300)

result = fleet_average_rate(cycle, fleet_2019, temp=25, humid_pct=50)
print(f"Fleet-average rate: {result.fleet_avg_g_per_mile:.1f} g/mile")
print(f"Fleet-average rate: {result.fleet_avg_g_per_km:.1f} g/km")

In [ ]:
# See the breakdown by vehicle configuration
top_contributors = result.by_config.nlargest(10, 'weighted_g_per_mile')
print("Top 10 contributors to fleet emissions:")
print(top_contributors[['source_type', 'fuel_type', 'nn_model_year', 'fraction', 'rate_g_per_mile', 'weighted_g_per_mile']].to_string(index=False))

### Effect of EV penetration

The `with_ev_penetration()` method creates a new fleet with a different EV share. EVs contribute zero tailpipe CO₂, so increasing EV share proportionally reduces the fleet average.

In [ ]:
# Compare fleet-average rate at different EV penetration levels
ev_levels = [0.0, 0.10, 0.20, 0.30, 0.50]
rates = []

for ev in ev_levels:
    fleet_ev = fleet_2019.with_ev_penetration(ev)
    res = fleet_average_rate(cycle, fleet_ev, temp=25, humid_pct=50)
    rates.append(res.fleet_avg_g_per_mile)
    print(f"  EV = {ev:5.0%} → fleet avg = {res.fleet_avg_g_per_mile:.1f} g/mile")

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot([e * 100 for e in ev_levels], rates, 'o-', color='teal', linewidth=2, markersize=8)
ax.set_xlabel('EV Penetration (%)', fontsize=11)
ax.set_ylabel('Fleet-Average CO\u2082 (g/mile)', fontsize=11)
ax.set_title('Fleet Emission Rate vs. EV Penetration', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 3. Annual Emissions (Layer 3)

Knowing g/mile is useful, but policy analysts need **annual totals in metric tons**. Layer 3 scales per-mile rates using Vehicle Miles Traveled (VMT) data from the MOVES database.

The VMT comes from the **Highway Performance Monitoring System (HPMS)**, the same source EPA MOVES uses. Monthly fractions allow seasonal breakdowns.

In [ ]:
from neuralmoves import VMTAllocation, annualize_emissions

# Load VMT data for Passenger Cars (source type 21), year 2019
vmt = VMTAllocation.from_moves_defaults(source_type_id=21, calendar_year=2019)
print(vmt)
print(f"\nAnnual VMT: {vmt.annual_vmt:,.0f} miles")

In [ ]:
# Annualize the fleet-average rate we computed earlier
annual = annualize_emissions(rate_g_per_mile=result.fleet_avg_g_per_mile, vmt=vmt)
print(f"Annual CO\u2082: {annual.annual_co2_g:,.0f} g")
print(f"Annual CO\u2082: {annual.annual_co2_metric_tons:,.0f} metric tons")

In [ ]:
# Monthly breakdown
monthly = annual.monthly_breakdown
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Monthly VMT
ax1.bar(month_names, monthly['monthly_vmt'] / 1e9, color='steelblue')
ax1.set_ylabel('VMT (billions)', fontsize=11)
ax1.set_title('Monthly VMT Distribution', fontsize=12, fontweight='bold')
ax1.tick_params(axis='x', rotation=45)

# Monthly CO2
ax2.bar(month_names, monthly['co2_g'] / 1e12, color='coral')
ax2.set_ylabel('CO\u2082 (million metric tons)', fontsize=11)
ax2.set_title('Monthly CO\u2082 Emissions', fontsize=12, fontweight='bold')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

---
## 4. EV Policy Scenarios (Layer 4)

Real policies don't set a fixed EV fraction — they ramp up adoption over time. A `TechnologyScenario` defines an EV adoption trajectory: for each year, what percentage of the fleet is electric?

NeuralMOVES supports three adoption curves:
- **Constant**: Fixed EV share (e.g., business-as-usual)
- **Linear ramp**: Steady year-over-year increase
- **S-curve**: Logistic adoption curve (slow start → rapid growth → saturation), which is more realistic for technology diffusion

In [ ]:
from neuralmoves import TechnologyScenario

# Define three scenarios
bau = TechnologyScenario.constant("Business as usual", ev_fraction=0.05)
linear = TechnologyScenario.linear_ev_ramp("Linear ramp", 2025, 2040, 0.05, 0.50)
scurve = TechnologyScenario.s_curve_ev_adoption("S-curve adoption", 2025, 2040, 0.05, 0.50, steepness=0.5)

print(bau)
print(linear)
print(scurve)

In [ ]:
# Visualize the adoption curves
years = list(range(2025, 2041))

fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(years, [bau.get_ev_fraction(y) * 100 for y in years], 's--', label='Business as usual', color='gray')
ax.plot(years, [linear.get_ev_fraction(y) * 100 for y in years], 'o-', label='Linear ramp', color='steelblue')
ax.plot(years, [scurve.get_ev_fraction(y) * 100 for y in years], '^-', label='S-curve adoption', color='teal')
ax.set_xlabel('Year', fontsize=11)
ax.set_ylabel('EV Fleet Share (%)', fontsize=11)
ax.set_title('EV Adoption Scenarios (2025-2040)', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

For any year, `get_fleet()` returns a `FleetComposition` with the corresponding EV share:

In [ ]:
# Get the fleet for 2030 under the linear ramp
fleet_2030 = linear.get_fleet(2030)
print(f"2030 fleet under linear ramp:")
print(f"  EV penetration: {fleet_2030.ev_penetration:.1%}")
print(f"  Vehicle configs: {fleet_2030.n_vehicle_configs}")

---
## 5. Scenario Comparison (Layer 6)

The top layer ties everything together: define complete scenarios (technology trajectory + years + optional cycle/location) and compare them side by side.

A `ScenarioDefinition` bundles all the parameters. `compare_scenarios()` evaluates each scenario year by year and returns a comparison table.

In [ ]:
from neuralmoves import ScenarioDefinition, compare_scenarios, evaluate_scenario

# Define two scenarios: business-as-usual vs. aggressive EV policy
years = list(range(2025, 2036))

baseline = ScenarioDefinition(
    name="BAU (5% EV)",
    technology=TechnologyScenario.constant("bau", ev_fraction=0.05),
    years=years,
)

policy = ScenarioDefinition(
    name="EV push (5-30%)",
    technology=TechnologyScenario.linear_ev_ramp("policy", 2025, 2035, 0.05, 0.30),
    years=years,
)

print(f"Baseline: {baseline.name}")
print(f"Policy:   {policy.name}")
print(f"Years:    {years[0]}-{years[-1]}")

In [ ]:
# Evaluate and compare
results = compare_scenarios([baseline, policy])
print(results[['year', 'BAU (5% EV)_co2_MT', 'EV push (5-30%)_co2_MT',
               'BAU (5% EV)_ev_pct', 'EV push (5-30%)_ev_pct']].to_string(index=False))

In [ ]:
# Visualize the comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# CO2 over time
ax1.plot(results['year'], results['BAU (5% EV)_co2_MT'], 's-', label='BAU (5% EV)', color='gray', linewidth=2)
ax1.plot(results['year'], results['EV push (5-30%)_co2_MT'], 'o-', label='EV push (5\u219230%)', color='teal', linewidth=2)
ax1.set_xlabel('Year', fontsize=11)
ax1.set_ylabel('Annual CO\u2082 (metric tons)', fontsize=11)
ax1.set_title('Annual Emissions by Scenario', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# EV fraction over time
ax2.plot(results['year'], results['BAU (5% EV)_ev_pct'] * 100, 's-', label='BAU', color='gray', linewidth=2)
ax2.plot(results['year'], results['EV push (5-30%)_ev_pct'] * 100, 'o-', label='EV push', color='teal', linewidth=2)
ax2.set_xlabel('Year', fontsize=11)
ax2.set_ylabel('EV Fleet Share (%)', fontsize=11)
ax2.set_title('EV Penetration by Scenario', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### EV impact analysis

How does the fleet emission rate change as you increase EV penetration from 0% to 50%? `ev_impact_analysis()` sweeps across EV fractions and reports the CO₂ reduction.

In [ ]:
from neuralmoves import ev_impact_analysis

ev_sweep = ev_impact_analysis(
    ev_fractions=[0.0, 0.05, 0.10, 0.15, 0.20, 0.30, 0.40, 0.50],
    calendar_year=2019,
)
print(ev_sweep.to_string(index=False))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(ev_sweep['ev_fraction'] * 100, ev_sweep['fleet_rate_g_per_mile'], 'o-', color='steelblue', linewidth=2)
ax1.set_xlabel('EV Penetration (%)', fontsize=11)
ax1.set_ylabel('Fleet Rate (g/mile)', fontsize=11)
ax1.set_title('Fleet Emission Rate', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2.plot(ev_sweep['ev_fraction'] * 100, ev_sweep['reduction_pct'], 'o-', color='teal', linewidth=2)
ax2.set_xlabel('EV Penetration (%)', fontsize=11)
ax2.set_ylabel('CO\u2082 Reduction (%)', fontsize=11)
ax2.set_title('CO\u2082 Reduction vs. Baseline', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 6. County-Level Analysis (Layer 5)

Emissions depend on local weather: cold winters increase CO₂ because engines run less efficiently. NeuralMOVES bundles monthly average temperature and humidity for **3,232 US counties** from the MOVES database.

A `LocationProfile` stores a county's meteorology and VMT data. You can then compute a monthly emission inventory that accounts for seasonal temperature variation.

In [ ]:
from neuralmoves import LocationProfile

# Load meteorology for Wayne County, MI (Detroit area)
wayne = LocationProfile.from_county_defaults(26163, location_name="Wayne County, MI")
print(wayne)
print(f"\nMonthly temperatures (\u00b0F):")
for month_id in range(1, 13):
    temp_f = wayne.avg_temp_by_month.get(month_id, 0)
    print(f"  Month {month_id:2d}: {temp_f:.1f}\u00b0F ({(temp_f - 32) * 5/9:.1f}\u00b0C)")

In [ ]:
# Load a contrasting warm-climate county
# Miami-Dade County, FL
miami = LocationProfile.from_county_defaults(12086, location_name="Miami-Dade, FL")

print(f"Wayne County, MI  — Jan: {wayne.avg_temp_by_month[1]:.0f}\u00b0F, Jul: {wayne.avg_temp_by_month[7]:.0f}\u00b0F")
print(f"Miami-Dade, FL    — Jan: {miami.avg_temp_by_month[1]:.0f}\u00b0F, Jul: {miami.avg_temp_by_month[7]:.0f}\u00b0F")

### Monthly emission inventory

`location_inventory()` evaluates the drive cycle at each month's local temperature and humidity, then scales by monthly VMT. This captures how cold winters increase emissions.

In [ ]:
from neuralmoves.geography import location_inventory

fleet = FleetComposition.from_moves_defaults(calendar_year=2019)
cycle = DriveCycle.constant_speed(speed_mph=30, duration_s=300)

inv_wayne = location_inventory(wayne, fleet, cycle)
inv_miami = location_inventory(miami, fleet, cycle)

print("Wayne County, MI monthly inventory:")
print(inv_wayne[['monthID', 'temperature_F', 'fleet_rate_g_per_mile', 'monthly_co2_metric_tons']].to_string(index=False))

In [ ]:
# Visualize: emission rate by month for both counties
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7), sharex=True)

# Temperature comparison
ax1.plot(month_names, inv_wayne['temperature_F'], 's-', label='Wayne Co., MI', color='steelblue', linewidth=2)
ax1.plot(month_names, inv_miami['temperature_F'], 'o-', label='Miami-Dade, FL', color='coral', linewidth=2)
ax1.set_ylabel('Temperature (\u00b0F)', fontsize=11)
ax1.set_title('Monthly Temperature and Fleet Emission Rate', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Emission rate comparison
ax2.plot(month_names, inv_wayne['fleet_rate_g_per_mile'], 's-', label='Wayne Co., MI', color='steelblue', linewidth=2)
ax2.plot(month_names, inv_miami['fleet_rate_g_per_mile'], 'o-', label='Miami-Dade, FL', color='coral', linewidth=2)
ax2.set_ylabel('Fleet Rate (g/mile)', fontsize=11)
ax2.set_xlabel('Month', fontsize=11)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 7. Putting It All Together

Let's answer a concrete policy question:

> **If Wayne County, MI ramps EV adoption from 5% to 30% between 2025 and 2035, how much CO₂ does that save compared to staying at 5%?**

This combines all layers: drive cycle evaluation (L1), fleet averaging (L2), VMT annualization (L3), technology trajectories (L4), county meteorology (L5), and scenario comparison (L6).

In [ ]:
wayne = LocationProfile.from_county_defaults(26163, location_name="Wayne County, MI")
years = list(range(2025, 2036))

# Baseline: 5% EV constant
baseline_wayne = ScenarioDefinition(
    name="Wayne BAU",
    technology=TechnologyScenario.constant("bau", ev_fraction=0.05),
    years=years,
    location=wayne,
)

# Policy: 5% -> 30% linear ramp
policy_wayne = ScenarioDefinition(
    name="Wayne EV push",
    technology=TechnologyScenario.linear_ev_ramp("policy", 2025, 2035, 0.05, 0.30),
    years=years,
    location=wayne,
)

# Evaluate both scenarios
baseline_result = evaluate_scenario(baseline_wayne)
policy_result = evaluate_scenario(policy_wayne)

print(f"{'Year':>6} {'BAU CO\u2082 (MT)':>14} {'Policy CO\u2082 (MT)':>16} {'Reduction':>10}")
print("-" * 50)
for _, brow in baseline_result.yearly.iterrows():
    year = int(brow['year'])
    prow = policy_result.yearly[policy_result.yearly['year'] == year].iloc[0]
    bco2 = brow['annual_co2_metric_tons']
    pco2 = prow['annual_co2_metric_tons']
    reduction = (1 - pco2 / bco2) * 100 if bco2 > 0 else 0
    print(f"{year:>6} {bco2:>14,.0f} {pco2:>16,.0f} {reduction:>9.1f}%")

total_saved = baseline_result.total_co2_metric_tons - policy_result.total_co2_metric_tons
pct_saved = total_saved / baseline_result.total_co2_metric_tons * 100
print(f"\nTotal CO\u2082 saved (2025-2035): {total_saved:,.0f} metric tons ({pct_saved:.1f}%)")

In [ ]:
# Final visualization
fig, ax = plt.subplots(figsize=(10, 5))

ax.fill_between(baseline_result.yearly['year'], baseline_result.yearly['annual_co2_metric_tons'],
                policy_result.yearly['annual_co2_metric_tons'], alpha=0.3, color='teal', label='CO\u2082 savings')
ax.plot(baseline_result.yearly['year'], baseline_result.yearly['annual_co2_metric_tons'],
        's-', color='gray', linewidth=2, label='BAU (5% EV)')
ax.plot(policy_result.yearly['year'], policy_result.yearly['annual_co2_metric_tons'],
        'o-', color='teal', linewidth=2, label='EV push (5\u219230%)')

ax.set_xlabel('Year', fontsize=11)
ax.set_ylabel('Annual CO\u2082 (metric tons)', fontsize=11)
ax.set_title(f'Wayne County, MI: CO\u2082 Savings from EV Adoption\n'
             f'Total savings: {total_saved:,.0f} MT ({pct_saved:.1f}%) over 2025-2035',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## Summary

This notebook demonstrated the six aggregation layers in NeuralMOVES:

| Layer | Module | What it does | Key function |
|-------|--------|-------------|---------------|
| 1 | `cycles` | Evaluate a drive cycle for one vehicle | `evaluate_cycle()` |
| 2 | `fleet` | Fleet-average across all vehicle types | `fleet_average_rate()` |
| 3 | `temporal` | Scale to annual totals via VMT | `annualize_emissions()` |
| 4 | `technology` | Model EV adoption over time | `TechnologyScenario` |
| 5 | `geography` | County-specific meteorology | `LocationProfile` |
| 6 | `scenarios` | Compare policy scenarios | `compare_scenarios()` |

Each layer builds on the ones below but can be used independently. Enter at whatever level matches your analysis needs.

For architecture details, see [docs/architecture.md](../docs/architecture.md). For per-second API usage, see the [comprehensive usage guide](comprehensive_usage_guide.ipynb).

### Citation

If you use NeuralMOVES in your research, please cite:

```
Ramirez-Sanchez et al. (2026). NeuralMOVES: A lightweight and microscopic
vehicle emission estimation model based on reverse engineering and surrogate learning.
Transportation Research Part C: Emerging Technologies.
```